In [1]:
!pip show lxml

Name: lxml
Version: 4.5.2
Summary: Powerful and Pythonic XML processing library combining libxml2/libxslt with the ElementTree API.
Home-page: https://lxml.de/
Author: lxml dev team
Author-email: lxml-dev@lxml.de
License: BSD
Location: c:\programdata\anaconda38\lib\site-packages
Requires: 
Required-by: 


In [52]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os
import json

url = 'http://www.podbbang.com/_m_api/podcasts/16255/episodes?offset=0&sort=pubdate:desc&episode_id=0&limit=8&with=summary&cache=0'

res = requests.get(url) #요청 보냄
print(res.status_code)
if res.ok:
    # lxml 파서를 사용해서 BeautfiulSoup 객체생성
    soup = BeautifulSoup(res.text,'lxml') 
    #print(type(p_tag))
    #<p> </p> 사이의 text를 추출함
    p_tag = soup.select('p')[0].text
    
    #str 타입의 텍스트를 dict 타입으로 변경하기 위해서 json 내부모듈 사용
    json_data = json.loads(p_tag)
#     print(type(json_data), json_data)
    
    #{data:[{},{},{}]}
    episode_length = len(json_data['data'])
#     print(len(json_data['data']),type(json_data['data']))
    episode_list = json_data['data']      
    
    
    for episode in episode_list:
        print('------')
        #에피소드의 제목
        title = episode['title']
        #에피소드 다운로드 url
        mp3_url = episode['enclosure']['url']
        print(title, mp3_url)
        
        #url 값이 있으면 다운로드 받아야 함
        if mp3_url:
            
            req_header = {
                'referer':'http://www.podbbang.com/ch/16255',
                'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'
            }
            dir_name = os.path.join('mp3','여바라')
            print(dir_name) #mp3\여바라
            
            #해당 디렉토리가 없으면
            if not os.path.isdir(dir_name):
                #디렉토리 생성
                os.makedirs(dir_name)
            
            #해당 url에서 파일명만 추출
            file_name=os.path.basename(mp3_url)
            #생성된 디렉토리명과 파일명을 합쳐주기
            file_name=dir_name+'/'+file_name
            
            #print(file_name)
            res=requests.get(mp3_url,headers=req_header) #요청보냄
            if res.ok:
                #response 객체에서 binary 데이터 추출
                mp3_bin = res.content
                #binary 데이터를 local 파일로 저장
                with open(file_name, 'wb')as file:
                    file.write(mp3_bin)
    
            print(res.status_code)
            
        print(episode)

200
------
46. 소소하게 여행중독 #2 : 문상건 작가의 아시아와 세이셸 https://file.ssenhosting.com/data1/pb_21610/yeovara46.mp3
mp3\여바라
mp3\여바라/yeovara46.mp3
200
{'id': 22816888, 'podcast': {'id': 16255}, 'title': '46. 소소하게 여행중독 #2 : 문상건 작가의 아시아와 세이셸', 'description': '문상건 작가의 인도여행 스토리에 이어\n파키스탄, 미얀마 등 이후 아시아 여정까지 함께 합니다.\n\n여기서 끝이 아니죠!\n내셔널지오그래픽과 기네스북이 선정한 세계에서 가장 아름다운 해변을 보유한\n세이셸에 대한 스토리 역시 놓치지 마세요~\n\n진행 : 시아, 밍\n시그널 : 류경호\n지원 : 마을생활전파소', 'image': None, 'enclosure': {'type': 'audio/mp3', 'url': 'https://file.ssenhosting.com/data1/pb_21610/yeovara46.mp3', 'length': 41461938}, 'duration': '00:43:06', 'cash': 0, 'like_count': 6, 'is_adult_only': False, 'is_free': True, 'is_paid': False, 'can_download': True, 'pay_method': 'cash', 'sell_end_at': None, 'published_at': '2019-01-02 20:33:36'}
------
45. 소소하게 여행중독 - 문상건 작가의 인도 여행기 https://file.ssenhosting.com/data1/pb_21610/yeovara45.mp3
mp3\여바라
mp3\여바라/yeovara45.mp3
200
{'id': 22810950, 'podcast': {'id': 16255}, 'title': '45. 소소하게 여행중독 - 문상건 작가의 인도 여행기', 'descri

### 팟빵 다운로드 1단계
* 팟빵 id와 제목을 함수의 인자로 받는다.
* 첫번째의 8개의 에피소드만 다운로드 받고 있다.
* 저장되는 mp3파일명을 에피소드의 타이틀로 지정

In [38]:
#문자열에 있는 특수문자를 제거하는 함수
def clean_text(text):
    import re
    text = text.replace("\n", "")
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]','', text)
    cleaned_text = cleaned_text.replace(" ",'')
    return cleaned_text

In [39]:
clean_text("김영철의 파워FM - 진짜 미국식 영어 810회 - 타일러의 진짜 미국식 영어 = Don't be so stubborn. = 억지 좀 부리지 마세요!!")

'김영철의파워FM진짜미국식영어810회타일러의진짜미국식영어Dontbesostubborn억지좀부리지마세요'

In [40]:
def podbbang_download(pod_id, title):
    import requests
    from bs4 import BeautifulSoup
    from urllib.parse import urljoin
    import os
    import json
    
    url = f'http://www.podbbang.com/_m_api/podcasts/{pod_id}/episodes'
    # 함수로 {pod_id}변수로 처리하기 쉬우려고 request_param사용 1.Nhn_번역_검색서비스 참고
    req_param={
        "offset":0,
        "sort":"pubdate:desc",
        "episode_id":0,
        "limit":8,
        "with":"summary",
        "cache":0

    }
    
    res = requests.get(url, params=req_param) #요청 보냄
    print(res.status_code)
    if res.ok:
        # lxml 파서를 사용해서 BeautfiulSoup 객체생성
        soup = BeautifulSoup(res.text,'lxml') 
        #print(type(p_tag))
        #<p> </p> 사이의 text를 추출함
        p_tag = soup.select('p')[0].text

        #str 타입의 텍스트를 dict 타입으로 변경하기 위해서 json 내부모듈 사용
        json_data = json.loads(p_tag)
    #     print(type(json_data), json_data)

        #{data:[{},{},{}]}
        episode_length = len(json_data['data'])
    #     print(len(json_data['data']),type(json_data['data']))
        episode_list = json_data['data']      

        for episode in episode_list:
            print('------')
            #에피소드의 제목
            episode_title = episode['title']
            #에피소드 다운로드 url
            mp3_url = episode['enclosure']['url']
            print(episode_title, mp3_url)

            #url 값이 있으면 다운로드 받아야 함
            if mp3_url:

                req_header = {
                    'referer':f'http://www.podbbang.com/ch/{pod_id}',
                    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'
                }
                dir_name = os.path.join('mp3',title)
                print(dir_name) #mp3\여바라
                #해당 디렉토리가 없으면
                if not os.path.isdir(dir_name):
                    #디렉토리 생성
                    os.makedirs(dir_name)

                #해당 url에서 파일명만 추출
                #file_name=os.path.basename(mp3_url)
                episode_title = clean_text(episode_title)
                file_name=f'{episode_title}.mp3'
                
                #생성된 디렉토리명과 파일명을 합쳐주기
                file_name=dir_name+'/'+file_name

                #print(file_name)
                res=requests.get(mp3_url,headers=req_header)
                if res.ok:
                    #response 객체에서 binary 데이터 추출
                    mp3_bin = res.content
                    #binary 데이터를 local 파일로 저장
                    with open(file_name, 'wb')as file:
                        file.write(mp3_bin)

                print(res.status_code)

            print(episode)
            

In [41]:
podbbang_download(13410,'김영철미국식영어')

200
------
김영철의 파워FM - 진짜 미국식 영어 810회 - 타일러의 진짜 미국식 영어 = Don't be so stubborn. = 억지 좀 부리지 마세요!! https://rss.art19.com/episodes/5314131e-25af-4b88-82a9-e810a5c6c282.mp3
mp3\김영철미국식영어
200
{'id': 23856711, 'podcast': {'id': 13410}, 'title': "김영철의 파워FM - 진짜 미국식 영어 810회 - 타일러의 진짜 미국식 영어 = Don't be so stubborn. = 억지 좀 부리지 마세요!!", 'description': "김영철의 파워FM - 진짜 미국식 영어 810회 - 타일러의 진짜 미국식 영어 = Don't be so stubborn. = 억지 좀 부리지 마세요!!\n\n\n", 'image': None, 'enclosure': {'type': 'audio/mpeg', 'url': 'https://rss.art19.com/episodes/5314131e-25af-4b88-82a9-e810a5c6c282.mp3', 'length': 5607340}, 'duration': '00:05:50', 'cash': 0, 'like_count': 8, 'is_adult_only': False, 'is_free': True, 'is_paid': False, 'can_download': True, 'pay_method': 'cash', 'sell_end_at': None, 'published_at': '2020-10-23 11:29:34'}
------
김영철의 파워FM - 진짜 미국식 영어 809회 - 타일러의 진짜 미국식 영어 = She's such a nice person. = 인심이 진짜 후하시네요! https://rss.art19.com/episodes/0680c432-7b4d-48ad-b552-c26bb723b4ca.mp3
mp3\김영철미국식영어
200
{'id': 2385284

### 팟빵 다운로드 2단계
* 팟빵 id와 제목을 함수의 인자로 받는다.
* 모든 에피소드를 다운로드 받고 있다.
* 저장되는 mp3파일명을 에피소드의 타이틀로 지정
 - 에피소드를 요청할 때 offset 값의 최대값이 항상 변하므로 
 - itertools의 count()함수를 사용해 무한루프를 수행하도록 한다.
 - 무한루프를 탈출할 수 있는 조건식을 반드시 작성
    - json_data.['data'].길이가 0 이면 무한루프를 탈출한다.
 - 사람이 직접 다운로드 받는 것처럼 보이게 하기 위해서 sleep time 설정한다.
     - time이란 내부모듈의 sleep함수 사용

In [45]:
def podbbang_all_download(pod_id, title):
    import requests
    from bs4 import BeautifulSoup
    from urllib.parse import urljoin
    import os
    import json
    from itertools import count
    from time import sleep
    
    url = f'http://www.podbbang.com/_m_api/podcasts/{pod_id}/episodes'
    
    for offset in count(0):
        
        req_param={
            "offset":offset,
            "sort":"pubdate:desc",
            "episode_id":0,
            "limit":8,
            "with":"summary",
            "cache":0
        }

        res = requests.get(url, params=req_param) #요청 보냄
        print(res.status_code)
        if res.ok:
            # lxml 파서를 사용해서 BeautfiulSoup 객체생성
            soup = BeautifulSoup(res.text,'lxml') 
            #print(type(p_tag))
            #<p> </p> 사이의 text를 추출함
            p_tag = soup.select('p')[0].text

            #str 타입의 텍스트를 dict 타입으로 변경하기 위해서 json 내부모듈 사용
            json_data = json.loads(p_tag)
        #     print(type(json_data), json_data)

            #{data:[{},{},{}]}
            episode_length = len(json_data['data'])
            print('offset:',offset,'data의 길이 => ', episode_length)
            #무한루프를 탈출하는 조건을 설정
            if episode_length == 0:
                return None
            
        #     print(len(json_data['data']),type(json_data['data']))
            episode_list = json_data['data']      

            for episode in episode_list:
                print('------')
                #에피소드의 제목
                episode_title = episode['title']
                #에피소드 다운로드 url
                mp3_url = episode['enclosure']['url']
                print(episode_title, mp3_url)

                #url 값이 있으면 다운로드 받아야 함
                if mp3_url:

                    req_header = {
                        'referer':f'http://www.podbbang.com/ch/{pod_id}',
                        'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'
                    }
                    dir_name = os.path.join('mp3',title)
                    print(dir_name) #mp3\여바라
                    
                    #해당 디렉토리가 없으면
                    if not os.path.isdir(dir_name):
                        #디렉토리 생성
                        os.makedirs(dir_name)

                    #해당 url에서 파일명만 추출
                    #file_name=os.path.basename(mp3_url)
                    episode_title = clean_text(episode_title)
                    file_name=f'{episode_title}.mp3'

                    #생성된 디렉토리명과 파일명을 합쳐주기
                    file_name=dir_name+'/'+file_name

                    #print(file_name)
                    res=requests.get(mp3_url,headers=req_header)
                    if res.ok:
                        #response 객체에서 binary 데이터 추출
                        mp3_bin = res.content
                        #binary 데이터를 local 파일로 저장
                        with open(file_name, 'wb')as file:
                            file.write(mp3_bin)
                            
                        #0.5초간 프로세스를 중지, 기계가 아니라 사람처럼 보이게 하려고
                        sleep(0.5)


In [46]:
podbbang_all_download(16255,'여바라')

200
offset: 0 data의 길이 =>  8
------
46. 소소하게 여행중독 #2 : 문상건 작가의 아시아와 세이셸 https://file.ssenhosting.com/data1/pb_21610/yeovara46.mp3
mp3\여바라
------
45. 소소하게 여행중독 - 문상건 작가의 인도 여행기 https://file.ssenhosting.com/data1/pb_21610/yeovara45.mp3
mp3\여바라
------
44. 여행직업탐구 : 국외여행인솔자 뽀개기 feat. 서주희 https://file.ssenhosting.com/data1/pb_21610/yeovara44.mp3
mp3\여바라
------
43. 코이카(KOICA) 뽀개기 feat.서주희 https://file.ssenhosting.com/data1/pb_21610/yeovara43.mp3
mp3\여바라
------
42. 6개월 동안의 신혼여행 feat 밤송이 https://file.ssenhosting.com/data1/pb_21610/yeovara42.mp3
mp3\여바라
------
춤추는 쥬디의 플래시몹 in 멕시코시티 https://file.ssenhosting.com/data1/pb_21610/mexico.mp4
mp3\여바라
------
41. 춤추는 쥬디의 멕시코 여행 https://file.ssenhosting.com/data1/pb_21610/yeovara41.mp3
mp3\여바라
------
40. 여행하다 배우다 : 남미에서 살사, 스페인어 정복하기 feat. 쥬디 https://file.ssenhosting.com/data1/pb_21610/yeovara40.mp3
mp3\여바라
200
offset: 1 data의 길이 =>  8
------
39. 불의 나라 아제르바이잔, 바람의 도시 바쿠 feat 바쿠소녀 송의영 https://file.ssenhosting.com/data1/pb_21610/yeovara39.mp3
mp3\여바라
------